## analysis pipeline for psd-slope

This notebook covers the whole analysis for the psd-slope project.

##### Jorge Yanar 2016
This notebook does the following:
1. Imports every .mat file in the specified directory
2. For each file, we run through channels and compute the PSD of each channel
3. Compute the average PSD for each subject
4. Use the RANSAC algorithm to fit a line to each PSD, using the PSD from 1 - 24 Hz but blocking out alpha, from 7 - 14 Hz.

##### Imports

In [1]:
%matplotlib inline
import os
import numpy as np
import scipy as sp
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import signal
from scipy import io
from sklearn import linear_model, datasets

##### Data importing

In [ ]:
def import_files()

##### Compute PSD

###### Fit PSD slope

##### Export results to CSV